In [7]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("face_mask_detection_model_2.h5")

# Print model summary to verify input shape
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,900 (12.61 MB)

 Trainable params: 3,304,898 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [18]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

from tensorflow.keras.optimizers import Adam

# Load the trained model
model = load_model("face_mask_detection_model_2.h5")
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Directory containing images for inference
inference_image_folder = "C:/Users/Hardik Nikam/Downloads/Dataset/Dataset/archive/Medical mask/Medical mask/Medical Mask/no annotaions"

# List to store inference results
inference_results = []

# Loop through the images in the folder
for img_name in os.listdir(inference_image_folder):
    if img_name.endswith(('.jpg', '.png')):
        img_path = os.path.join(inference_image_folder, img_name)
        img = cv2.imread(img_path)
        
        if img is not None:
            # Convert to grayscale for face detection
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

            # Loop through each detected face
            for (x, y, w, h) in faces:
                # Get bounding box coordinates
                x1, y1, x2, y2 = x, y, x + w, y + h

                # Extract the face region for classification
                face_region = img[y:y + h, x:x + w]
                face_resized = cv2.resize(face_region, (126, 126))  # Resize to model's input shape
                face_normalized = face_resized.astype('float32') / 255.0
                face_expanded = np.expand_dims(face_normalized, axis=0)  # Add batch dimension

                # Run inference on the face region
                predictions = model.predict(face_expanded, verbose=0)
                predicted_class = np.argmax(predictions[0])  # 0 or 1
                
                # Append results based on predicted class
                class_name = "face_with_mask" if predicted_class == 1 else "face_no_mask"
                inference_results.append([img_name, x1, x2, y1, y2, class_name])

# Convert results to a DataFrame
inference_df = pd.DataFrame(inference_results, columns=['name', 'x1', 'x2', 'y1', 'y2', 'classname'])

# Save the results to a CSV file
inference_df.to_csv("inference_results_4.csv", index=False)


In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

# Load the face mask detection model
model = tf.keras.models.load_model('face_mask_detection_model_2.h5')

# Set the input shape required by the model
input_shape = (128, 128, 3)  # Based on the model summary

# Load OpenCV's pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Directory containing images for inference
inference_image_folder = "C:/Users/Hardik Nikam/Downloads/Dataset/Dataset/archive/Medical mask/Medical mask/Medical Mask/no annotaions"

# List to store inference results
inference_results = []

# Loop through the images in the folder
for img_name in os.listdir(inference_image_folder):
    if img_name.endswith(('.jpg', '.png')):  # Check for image files
        img_path = os.path.join(inference_image_folder, img_name)
        image = cv2.imread(img_path)
        
        if image is not None:
            orig_image = image.copy()
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Detect faces in the image
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=2, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

            # Check if faces are detected
            if len(faces) == 0:
                print(f"No faces detected in {img_name}.")
                continue

            # Loop over detected faces
            for (x, y, w, h) in faces:
                # Extract face region of interest (ROI)
                face_roi = image[y:y + h, x:x + w]
                face_roi = cv2.resize(face_roi, (input_shape[0], input_shape[1]))  # Resize to 128x128
                face_roi = face_roi / 255.0  # Normalize pixel values
                face_roi = np.expand_dims(face_roi, axis=0)  # Add batch dimension (1, 128, 128, 3)

                # Predict mask or no mask
                prediction = model.predict(face_roi)[0]
                class_name = np.argmax(prediction)  # Get the class index with the highest probability
                
                # Append results
                inference_results.append({
                    'name': img_name,
                    'x1': x,
                    'y1': y,
                    'x2': x + w,
                    'y2': y + h,
                    'classname': "face_with_mask" if class_name == 1 else "face_no_mask"
                })

                # Draw bounding box and label on the image for visualization (optional)
                label = 'face_with_mask' if class_name == 1 else 'face_no_mask'  # Assuming 1 is mask, 0 is no mask
                color = (0, 255, 0) if class_name == 1 else (0, 0, 255)
                cv2.rectangle(orig_image, (x, y), (x + w, y + h), color, 2)
                cv2.putText(orig_image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            # Save the output image with bounding boxes (optional)
            output_image_path = os.path.join("output_images", f'output_{img_name}')
            cv2.imwrite(output_image_path, orig_image)

# Convert results to a DataFrame
inference_df = pd.DataFrame(inference_results)

# Save the results to a CSV file
inference_df.to_csv("inference_results_6.csv", index=False)

print("Inference complete. Results saved to inference_results.csv and output images in the output_images folder.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
No faces detected in 0003.jpg.
No faces detected in 0004.jpg.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
No faces detected in 0010.jpg.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
No faces detected in 0014.jpg.
No faces detected in 0015.jpg.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
No faces detected in 0018.jpg.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━